In [1]:
!pip install numpy pandas tensorflow scikit-learn pillow matplotlib seaborn scipy

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/07/5f/63760ff107bcf5146eee41b38b3985f9055e710a72fdd637b791dea3495c/pandas-2.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/26/92/85ec172418f39474c1cd0221d611345d4f433fc4ee2fc68e01f524ccc4e4/scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/bb/ba/63f402e74875486b87ec6506a4f93f6d8a0d94d10467280f3d9d7837ce3a/scipy-1.16.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.m

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
import multiprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.utils import shuffle
from PIL import ImageFile
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import Callback

2025-08-04 20:06:42.031128: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-04 20:06:42.031170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-04 20:06:42.031211: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-04 20:06:42.039434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Clear the previous session
K.clear_session()

# File paths
style_train = '/workspace/wikiart_extracted/wikiart/new_style_train_final.csv'
style_val = '/workspace/wikiart_extracted/wikiart/new_style_val_final.csv'
image_directory = '/workspace/wikiart_extracted/wikiart'

# Load datasets
train_df = pd.read_csv(style_train)
val_df = pd.read_csv(style_val)

# Combine datasets
all_df = pd.concat([train_df, val_df], axis=0).reset_index(drop=True)

print(f"Before encoding: {len(all_df)} images")
all_labels = all_df['style_name'].str.get_dummies(sep='/')
print(f"After encoding: {len(all_labels)} labels (should match images)")

# Filter out styles with too few training samples
min_train_samples = 500
valid_classes = all_labels.sum(axis=0)[all_labels.sum(axis=0) >= min_train_samples].index
all_labels = all_labels[valid_classes]
all_df = all_df[all_labels.sum(axis=1) > 0]

# Ensure consistency between all_df and all_labels
all_labels = all_labels.loc[all_df.index].reset_index(drop=True)
all_df = all_df.reset_index(drop=True)

# Step 1: Split data - 10% Test Set
df_train_val, test_df, labels_train_val, test_labels = train_test_split(
    all_df, all_labels, test_size=0.1, random_state=42
)

# Step 2: Split remaining 90% into 70% Train and 20% Validation
train_df, val_df, train_labels, val_labels = train_test_split(
    df_train_val, labels_train_val, test_size=2/9, random_state=42
)

# Image size and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 256

# Hybrid Sampling Function
def hybrid_sampling(df, labels):
    min_samples_per_class = 2000  # Only upsample classes below this
    max_samples_per_class = 3000  # Prevent large classes from growing

    image_counts = {}  # Track how often each image appears across all labels
    df_resampled = []
    labels_resampled = []

    # 1️⃣ **Ensure ONLY small classes (< 1200) get upsampled**
    for style in labels.columns:
        class_indices = labels[labels[style] == 1].index
        num_samples = len(class_indices)

        if num_samples < min_samples_per_class:
            resampled_indices = np.random.choice(class_indices, size=min_samples_per_class, replace=True)
        elif num_samples > max_samples_per_class:
            resampled_indices = np.random.choice(class_indices, size=max_samples_per_class, replace=False)
        else:
            resampled_indices = class_indices  # Keep mid-sized classes unchanged

        for idx in resampled_indices:
            img_path = df.loc[idx, 'image_path']

            # 2️⃣ **Prevent images from being overused**
            if img_path not in image_counts:
                image_counts[img_path] = 0
            image_counts[img_path] += 1

            if image_counts[img_path] <= 10:  # Allow max 2 occurrences per image
                df_resampled.append(df.loc[idx])
                labels_resampled.append(labels.loc[idx])

    # 3️⃣ **Convert lists back to DataFrame after resampling**
    resampled_df = pd.DataFrame(df_resampled).reset_index(drop=True)
    resampled_labels = pd.DataFrame(labels_resampled, columns=labels.columns).reset_index(drop=True)

    return shuffle(resampled_df, resampled_labels, random_state=42)

# Data Generators
def generator_with_labels(datagen, dataframe, labels, batch_size):
    num_samples = len(dataframe)
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]
            batch_images = []
            for idx in batch_indices:
                img_path = dataframe.iloc[idx]['image_path']
                img = tf.keras.preprocessing.image.load_img(f"{image_directory}/{img_path}", target_size=IMAGE_SIZE)
                img = tf.keras.preprocessing.image.img_to_array(img)
                img = datagen.random_transform(img)
                img = datagen.standardize(img)
                batch_images.append(img)
            batch_images = np.array(batch_images)
            batch_labels = labels.iloc[batch_indices].values
            yield batch_images, batch_labels

# Define Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Apply sampling to check actual image and label counts 
sampled_df, sampled_labels = hybrid_sampling(train_df, train_labels)

# Print class distribution after sampling
print("\nClass counts after resampling:")
print(sampled_labels.sum().sort_values(ascending=False))

# Print total number of training samples
print(f"\nTotal resampled training samples: {len(sampled_df)}")

Before encoding: 81446 images
After encoding: 81446 labels (should match images)

Class counts after resampling:
Realism_and_19th_Century_Movements      19000
Renaissance_and_Mannerism               11000
Impressionism_and_Post_Impressionism    10980
Expressionism_and_Derivatives           10001
Modern_Art                               9000
Baroque_and_Rococo                       8000
Baroque                                  5079
Impressionism                            4991
Expressionism                            4892
Realism                                  4049
Post_Impressionism                       3935
Romanticism                              3722
Symbolism                                3435
Art_Nouveau                              3421
Northern_Renaissance                     3198
Rococo                                   2921
Color_Field_Painting                     2920
Pop_Art                                  2851
Abstract_Expressionism                   2821
Minimalism   

In [4]:
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Multiply, Reshape
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K


# === Resample training set ===
train_df_resampled, train_labels_resampled = hybrid_sampling(train_df, train_labels)

# === Create generators ===
train_gen = generator_with_labels(train_datagen, train_df_resampled, train_labels_resampled, BATCH_SIZE)
val_gen = generator_with_labels(val_datagen, val_df, val_labels, BATCH_SIZE)

# === Compute steps ===
steps_per_epoch = int(np.ceil(len(train_df_resampled) / BATCH_SIZE))
validation_steps = int(np.ceil(len(val_df) / BATCH_SIZE))

# === Learning rate schedule ===
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=steps_per_epoch * 10,
    decay_rate=0.96,
    staircase=True
)

L2_FACTOR = 0.001
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[-100:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)  # shape: (None, 2048)
x = Dense(1024, activation='relu', kernel_regularizer=l2(L2_FACTOR))(x)  # (None, 1024)

def focal_loss(gamma=1.5, alpha=0.35):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')  # <-- Add this line
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(weight * cross_entropy)
    return loss


# === Gating mechanism: one gate per style ===
num_styles = train_labels.shape[1]
gates = Dense(num_styles * 1024, activation='sigmoid', name='gating_layer')(x)
gates = Reshape((num_styles, 1024))(gates)        # shape: (None, num_styles, 2048)
x_repeated = tf.keras.layers.RepeatVector(num_styles)(x)  # shape: (None, num_styles, 2048)
x_filtered = Multiply()([x_repeated, gates])      # shape: (None, num_styles, 2048)

# === Final classifier ===
logits = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(L2_FACTOR))
outputs = tf.keras.layers.TimeDistributed(logits)(x_filtered)  # shape: (None, num_styles, 1)
style_output = tf.keras.layers.Reshape((num_styles,))(outputs)

model = Model(inputs=base_model.input, outputs=style_output)

# === Compile model ===
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=focal_loss(gamma=1.5, alpha=0.35),
    metrics=[
        'accuracy',
        AUC(name='auc', multi_label=True),
        Precision(name='precision'),
        Recall(name='recall')
    ]
)

class SaveEachEpoch(Callback):
    def __init__(self, save_path):
        super().__init__()
        self.save_path = save_path
        os.makedirs(self.save_path, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        path = os.path.join(self.save_path, f'model_epoch_{epoch:02d}.h5')
        self.model.save(path)
        print(f'\nSaved model at {path}')

save_callback = SaveEachEpoch(save_path='checkpoints_final_focal_testing/')

# === Train model ===
history = model.fit(
    train_gen,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    verbose=1,
    callbacks=[save_callback]
)


2025-08-03 20:49:28.091785: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46866 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:2a:00.0, compute capability: 8.6


94765736/94765736 [==============================] - 3s 0us/step


2025-08-03 20:49:33.303600: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/30


2025-08-03 20:49:47.182466: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2025-08-03 20:49:47.270121: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-08-03 20:49:51.575583: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f947da775b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-03 20:49:51.575631: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-08-03 20:49:51.580190: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-03 20:49:51.690345: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

 80/274 [=======>......................] - ETA: 10:40 - loss: 1.2869 - accuracy: 0.1358 - auc: 0.5618 - precision: 0.0988 - recall: 0.0150

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3157: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


274/274 [==============================] - ETA: 0s - loss: 0.9418 - accuracy: 0.1904 - auc: 0.8055 - precision: 0.6584 - recall: 0.1289

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Saved model at checkpoints_final_focal_testing/model_epoch_00.h5
274/274 [==============================] - 1050s 4s/step - loss: 0.9418 - accuracy: 0.1904 - auc: 0.8055 - precision: 0.6584 - recall: 0.1289 - val_loss: 0.5920 - val_accuracy: 0.0277 - val_auc: 0.5434 - val_precision: 0.0726 - val_recall: 0.0371
Epoch 2/30
274/274 [==============================] - ETA: 0s - loss: 0.3112 - accuracy: 0.2472 - auc: 0.9402 - precision: 0.8558 - recall: 0.3471
Saved model at checkpoints_final_focal_testing/model_epoch_01.h5
274/274 [==============================] - 999s 4s/step - loss: 0.3112 - accuracy: 0.2472 - auc: 0.9402 - precision: 0.8558 - recall: 0.3471 - val_loss: 0.1929 - val_accuracy: 0.0861 - val_auc: 0.7214 - val_precision: 0.2700 - val_recall: 0.0853
Epoch 3/30
274/274 [==============================] - ETA: 0s - loss: 0.0998 - accuracy: 0.2715 - auc: 0.9562 - precision: 0.8774 - recall: 0.4377
Saved model at checkpoints_final_focal_testing/model_epoch_02.h5
274/274 [========

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
import tensorflow.keras.backend as K
import scipy

# === Loss function used during training ===
def focal_loss(gamma=1.5, alpha=0.35):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(weight * cross_entropy)
    loss.__name__ = 'focal_loss'
    return loss

# === Validation generator ===
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_directory,
    x_col='image_path',
    y_col=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)
val_steps = int(np.ceil(len(val_df) / BATCH_SIZE))

# === Evaluate selected checkpoints ===
EVALUATE_EPOCHS = list(range(30))  # Modify as needed
checkpoint_dir = "/workspace/checkpoints_final_focal_testing/"
threshold = 0.3  # Modify as needed

for epoch in EVALUATE_EPOCHS:
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch:02d}.h5")
    if not os.path.exists(checkpoint_path):
        print(f"Skipped epoch {epoch} (not found)")
        continue

    print(f"\n=== Evaluating Epoch {epoch+1} ===")
    
    model = load_model(checkpoint_path, custom_objects={'loss': focal_loss(gamma=1.5, alpha=0.35)})

    val_predictions = model.predict(val_generator, steps=val_steps)
    val_predictions = val_predictions[:len(val_df)]
    val_predicted_labels = (val_predictions > threshold).astype(int)

    val_auc = roc_auc_score(val_labels, val_predictions, average='macro')
    val_f1 = f1_score(val_labels, val_predicted_labels, average='macro', zero_division=0)
    val_precision = precision_score(val_labels, val_predicted_labels, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_predicted_labels, average='macro', zero_division=0)

    print(f"Validation AUC: {val_auc:.4f}")
    print(f"Validation F1-Score: {val_f1:.4f}")
    print(f"Validation Precision: {val_precision:.4f}")
    print(f"Validation Recall: {val_recall:.4f}")


Found 16290 validated image filenames.

=== Evaluating Epoch 1 ===


2025-08-04 20:06:52.032309: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46866 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:43:00.0, compute capability: 8.6
2025-08-04 20:06:54.311451: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-08-04 20:07:00.564202: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2025-08-04 20:07:00.681740: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


64/64 [==============================] - 116s 2s/step
Validation AUC: 0.5451
Validation F1-Score: 0.0121
Validation Precision: 0.1076
Validation Recall: 0.0740

=== Evaluating Epoch 2 ===
64/64 [==============================] - 106s 2s/step
Validation AUC: 0.7218
Validation F1-Score: 0.1262
Validation Precision: 0.1872
Validation Recall: 0.2137

=== Evaluating Epoch 3 ===
64/64 [==============================] - 107s 2s/step
Validation AUC: 0.8689
Validation F1-Score: 0.3521
Validation Precision: 0.5084
Validation Recall: 0.4090

=== Evaluating Epoch 4 ===
64/64 [==============================] - 105s 2s/step
Validation AUC: 0.9292
Validation F1-Score: 0.5129
Validation Precision: 0.4708
Validation Recall: 0.6316

=== Evaluating Epoch 5 ===
64/64 [==============================] - 106s 2s/step
Validation AUC: 0.9263
Validation F1-Score: 0.4947
Validation Precision: 0.4438
Validation Recall: 0.6433

=== Evaluating Epoch 6 ===
64/64 [==============================] - 107s 2s/step
Valida

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import (
    f1_score, roc_auc_score, precision_score, recall_score,
    classification_report, multilabel_confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as K

# === Define focal loss (must match training) ===
def focal_loss(gamma=1.5, alpha=0.35):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(weight * cross_entropy)
    loss.__name__ = 'focal_loss'
    return loss

# === Setup ===
threshold = 0.32
checkpoint_dir = "/workspace/checkpoints_final_focal_testing/"
EVALUATE_EPOCHS = list(range(30))

# === Test generator ===
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=image_directory,
    x_col='image_path',
    y_col=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)
test_steps = int(np.ceil(len(test_df) / BATCH_SIZE))

# === Evaluate each model ===
for epoch in EVALUATE_EPOCHS:
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch:02d}.h5")
    if not os.path.exists(checkpoint_path):
        print(f"Skipped epoch {epoch} (not found)")
        continue

    print(f"\n=== Evaluating Epoch {epoch + 1} on Test Set ===")

    model = load_model(checkpoint_path, custom_objects={'loss': focal_loss(gamma=1.5, alpha=0.35)})

    test_predictions = model.predict(test_generator, steps=test_steps)
    test_predictions = test_predictions[:len(test_df)]
    test_predicted_labels = (test_predictions > threshold).astype(int)

    # === Metrics ===
    test_auc = roc_auc_score(test_labels, test_predictions, average='macro')
    test_f1 = f1_score(test_labels, test_predicted_labels, average='macro', zero_division=0)
    test_precision = precision_score(test_labels, test_predicted_labels, average='macro', zero_division=0)
    test_recall = recall_score(test_labels, test_predicted_labels, average='macro', zero_division=0)

    print(f"Test AUC: {test_auc:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")

    print("\nClassification Report:")
    print(classification_report(
        test_labels, 
        test_predicted_labels, 
        target_names=valid_classes, 
        zero_division=0
    ))

Found 8145 validated image filenames.

=== Evaluating Epoch 1 on Test Set ===
32/32 [==============================] - 57s 2s/step
Test AUC: 0.5424
Test F1-Score: 0.0095
Test Precision: 0.0732
Test Recall: 0.0720

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.00      0.00      0.00       246
                         Art_Nouveau       0.00      0.00      0.00       425
                             Baroque       0.00      0.00      0.00       400
                  Baroque_and_Rococo       0.00      0.00      0.00       615
                Color_Field_Painting       0.03      0.01      0.02       144
                              Cubism       0.00      0.00      0.00       230
                   Early_Renaissance       0.00      0.00      0.00       139
                       Expressionism       0.14      0.00      0.00       685
       Expressionism_and_Derivatives       0.33      0.03   